In [1]:
%pip install little_mallet_wrapper

In [2]:
%pip install tomotopy

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install pyldavis

  Using cached pyLDAvis-3.4.0-py3-none-any.whl (2.6 MB)
  Using cached scikit_learn-1.2.2-cp39-cp39-win_amd64.whl (8.4 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached funcy-2.0-py2.py3-none-any.whl (30 kB)
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: python-dateutil>=2.7.3 in c:\users\mercu\anaconda3\lib\site-packages (from pandas>=1.3.4->pyldavis) (2.8.2)



ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mercu\\anaconda3\\Lib\\site-packages\\~klearn\\datasets\\_svmlight_format_fast.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2


In [3]:
%pip install gensim==3.8.3

In [2]:
#Comparing the LDA Implementations of Gensim, Mallet, and Tomotopy
#Dr. Zachary Stine
#Jake Davis

import nltk
nltk.download("stopwords")

#python library imports
import numpy as np
import json
import glob
import logging

#gensim imports
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#mallet imports
import os
import little_mallet_wrapper as lmw

#tomotopy imports
import tomotopy as tp
from tomotopy.utils import Corpus

#spacy
import spacy
from nltk.corpus import stopwords

#vis (May use for visualization)
#import pyLDAvis
#import pyLDAvis.gensim_models

#May replace with original stopwords at some point
stopwords = stopwords.words("english")
#stopwords.extend(['example', 'words'])

#Path to file to be read
training_data_path = "C:/Users/mercu/Downloads/ushmm_dn.json"

#Preparing the Data-----------------------------------------------------------
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mercu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
#import the designated corpus here.
data = load_data(training_data_path)["texts"]

def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc)) 
   if word not in stopwords] for doc in texts]

data_words = gen_words(data)
data_words_nostops = remove_stopwords(data_words)
print (data_words_nostops[0][0:20])

id2word = corpora.Dictionary(data_words_nostops)
texts = data_words_nostops
corpus = [id2word.doc2bow(text) for text in texts]
#print(corpus[:4]) #it will print the corpus we created above.

#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:4]] 
#it will print the words with their frequencies.

['name', 'david', 'kochalski', 'born', 'small', 'town', 'called', 'born', 'may', 'well', 'hard', 'working', 'six', 'children', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat']


In [4]:
print (data_words_nostops[1])

['right', 'name', 'rita', 'kerner', 'hilton', 'born', 'warsaw', 'july', 'lived', 'warsaw', 'six', 'years', 'old', 'seven', 'years', 'old', 'moved', 'mother', 'hometown', 'pomortsy', 'near', 'lodz', 'exiled', 'city', 'grandfather', 'practicing', 'dentist', 'since', 'mother', 'dentist', 'decided', 'practice', 'together', 'lived', 'war', 'broke', 'ghettos', 'small', 'girl', 'parents', 'well', 'financially', 'father', 'entrepreneur', 'mother', 'working', 'time', 'always', 'maid', 'nanny', 'small', 'moved', 'pomortsy', 'grandmother', 'running', 'house', 'always', 'sleep', 'help', 'since', 'mother', 'worked', 'office', 'apartment', 'babysitter', 'whoever', 'part', 'time', 'governess', 'come', 'school', 'hours', 'homework', 'read', 'take', 'park', 'kind', 'pampered', 'existence', 'went', 'sleep', 'away', 'camp', 'bicycle', 'know', 'expensive', 'loving', 'pampered', 'existence', 'time', 'grandfather', 'strongly', 'believed', 'reformed', 'judaism', 'exist', 'poland', 'never', 'kept', 'kosher', 

In [10]:
gensim_lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=5,
                                            alpha="auto")

In [12]:
gensim_lda_model.print_topic(1)


'0.017*"um" + 0.004*"yah" + 0.003*"bucharest" + 0.002*"bac_u" + 0.001*"ploie_ti" + 0.001*"jenine" + 0.001*"canteens" + 0.001*"unescu" + 0.001*"know" + 0.001*"legionnaires"'

In [7]:
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})
mallet_path = "C:/mallet-2.0.8/bin/mallet"

mallet_lda_model = gensim.models.wrappers.LdaMallet(
   mallet_path, corpus=corpus, num_topics=20, id2word=id2word
)

In [9]:
mallet_lda_model.print_topic(1)

'0.019*"jews" + 0.018*"polish" + 0.011*"town" + 0.011*"jewish" + 0.010*"poland" + 0.009*"jew" + 0.009*"germans" + 0.008*"told" + 0.008*"started" + 0.007*"german"'

In [72]:
#Go back to this if downgrading to gensim 3.8.3 is not a viable, futureproof option:
os.environ.update({'MALLET_HOME':r'C:/mallet-2.0.8/'})
mallet_path = "C:/mallet-2.0.8/bin/mallet"
mallet_output_path = "C:/users/mercu/Downloads"

mallet_lda_model = lmw.quick_train_topic_model(path_to_mallet=mallet_path,
                             output_directory_path=mallet_output_path, 
                             training_data=data, 
                             num_topics=20)

Importing data...
Complete
Training topic model...
Complete


In [7]:
tomotopy_lda_model = tp.LDAModel(k=20)
for vec in texts: 
    tomotopy_lda_model.add_doc(vec)

tomotopy_lda_model.burn_in = 100
tomotopy_lda_model.train(0)
print('Num docs:', len(tomotopy_lda_model.docs), ', Vocab size:', tomotopy_lda_model.num_vocabs, ', Num words:', tomotopy_lda_model.num_words)

Num docs: 252 , Vocab size: 28905 , Num words: 1037693


In [8]:
iter=50
for i in range(0, iter, 5):
    tomotopy_lda_model.train(50)

In [14]:
tomotopy_lda_model.get_topic_words(topic_id=2)

[('going', 0.03661682829260826),
 ('us', 0.03409622609615326),
 ('came', 0.027329297736287117),
 ('people', 0.024212535470724106),
 ('like', 0.02417069859802723),
 ('took', 0.017403770238161087),
 ('used', 0.017215508967638016),
 ('germans', 0.015709422528743744),
 ('away', 0.01511326339095831),
 ('could', 0.010825100354850292)]